In [37]:
using FileIO
using CSV
using XLSX
using DataFrames
using KnetMetrics
using RCall

In [38]:
#Import data as excel:
dirpath = "S:\\Sync\\University\\2023_MRP_1\\" #"/mnt/s/Sync/University/2023_MRP_1/"
filename =   "Annot_full.xlsx" # # "Annot_pilot.xlsx"
sheetname = "Annotations"
dataloc_on_sheet = "B3:AD4923" # "B3:AG78" # #Must include header row!!

#df = DataFrame(XLSX.readtable(dirpath*filename, sheetname))
xlf_mat = XLSX.readdata(dirpath*filename, sheetname*"!"*dataloc_on_sheet)

colnames = xlf_mat[1,:]
data = xlf_mat[2:end,:]
header = Symbol.(colnames[:])

data[.|(data .== nothing, string.(data) .== "-1", string.(data) .== "-9", data .== "~")] .= missing

#data[data .== "~"] .= missing
@show size(data)

df = DataFrame(data, header)

filter!(row -> row["selected_for_check"] == 1, df)
@show size(df)
@show header

size(data) = (4920, 29)
size(df) = 

(737, 29)
header = [Symbol("c+B3:W348mt_id"), Symbol("batch_#"), Symbol("cmt_#_in_Batch"), :comment_body, :engagement_human, :sentiment_human, :engagement_GPT4_A, :engagement_GPT4_B, :engagement_GPT4_C, :engagement_GPT4_D, :engagement_GPT4_E, :engagement_GPT4_F, :engagement_GPT4_G, :engagement_GPT4_H, :sentiment_GPT4_A, :sentiment_GPT4_B, :sentiment_GPT4_C, :sentiment_GPT4_D, :sentiment_GPT4_E, :sentiment_GPT4_F, :sentiment_GPT4_G, :sentiment_GPT4_H, :engagement_GPT4_majority, :sentiment_GPT4_majority, :engagement_maj_agreement, :sentiment_maj_agreement, :article_id, :article_headline, :selected_for_check]


29-element Vector{Symbol}:
 Symbol("c+B3:W348mt_id")
 Symbol("batch_#")
 Symbol("cmt_#_in_Batch")
 :comment_body
 :engagement_human
 :sentiment_human
 :engagement_GPT4_A
 :engagement_GPT4_B
 :engagement_GPT4_C
 :engagement_GPT4_D
 ⋮
 :sentiment_GPT4_G
 :sentiment_GPT4_H
 :engagement_GPT4_majority
 :sentiment_GPT4_majority
 :engagement_maj_agreement
 :sentiment_maj_agreement
 :article_id
 :article_headline
 :selected_for_check

In [39]:
#package1 = "irr"
#@rput package1
#R"install.packages(package1)"
#package2 = "psych"
#@rput package2
#R"install.packages(package2)"

R"library(irr)"
R"library(psych)"


Human_cols = filter(cn -> endswith(String(cn), "_human"), header)
#Human_cols = filter(cn -> endswith(String(cn), "_human_final"), header)
GPT4_cols = Dict(gpt_inst_label => filter(cn -> endswith(String(cn), "GPT4_$gpt_inst_label"), header) for gpt_inst_label in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
GPT4_major_cols = filter(cn -> endswith(String(cn), "GPT4_majority"), header)
var_types = fill("nominal", 2) #Column .. to .. only

function irr_2rater(A_cols, B_cols)
    dinfo1 = nothing
    dinfo2 = nothing
    dinfo3 = nothing
    dinfo4 = []

    cohenkappa_outputs = []
    percentagree_outputs = []
    count_outputs = []
    agreementCount_outputs = []
    for (c_var_type, col_A, col_B) in zip(var_types, A_cols, B_cols)
        c_var_data_df = df[:, [col_A, col_B]]
        
        c_var_data = Matrix(c_var_data_df)
        
        @rput c_var_data
        @rput c_var_type
        #dinfo1 = c_var_data
        R"c_var_data <- t(matrix(unlist(c_var_data), byrow=FALSE, ncol=2))"
        #R"dinfo2 <- c_var_data"
        #@rget dinfo2

        c_var_data = c_var_data_df[completecases(c_var_data_df), :] #Remove missin balues


        percent_agree = 100 * count(c_var_data[:, 1] .== c_var_data[:, 2]) / size(c_var_data)[1]
        push!(count_outputs,  size(c_var_data)[1])
        push!(agreementCount_outputs, count(c_var_data[:, 1] .== c_var_data[:, 2]))
        push!(percentagree_outputs, percent_agree)
        
        if prod(size(c_var_data)) == 0 #If all cases missing
            cohenkappa_output = Dict(:value=>missing)
            push!(cohenkappa_outputs, cohenkappa_output)
        else
            #push!(dinfo4, size(c_var_data))
            @rput c_var_data
            R"c_var_data <- t(matrix(unlist(c_var_data), byrow=FALSE, ncol=2))"
            weight_mode = "unweighted"
            @rput weight_mode

            R"cohenkappa_output <- kappa2(t(c_var_data), weight = weight_mode, sort.levels = FALSE)"
            #R"cohenkappa_output <- cohen.kappa(as.data.frame(t(c_var_data)))"
            @rget cohenkappa_output
            push!(cohenkappa_outputs, cohenkappa_output)
            
        end
        #View confusion matrix
        #c = confusion_matrix(c_var_data[:, 1],c_var_data[:, 2], labels=[0,1])
    
    end
    return cohenkappa_outputs, percentagree_outputs
end

#Start with Cohen's Kappa of the human vs. GPT4 majority vote
cohenkappa_outputs, percentagree_outputs = irr_2rater(Human_cols, GPT4_major_cols)

#Then averaged cohenkappa:
m_raters = 8
averaged_cohenkappa_outputs, averaged_percentagree_outputs = [0.0, 0.0],[0.0, 0.0]
for GPT4_inst_cols in values(GPT4_cols)
    current_kappas, current_percentagrees = irr_2rater(Human_cols, GPT4_inst_cols)
    for (i, c_kappa) in enumerate(current_kappas)
        averaged_cohenkappa_outputs[i] += c_kappa[:value] / m_raters
    end
    for (i, c_percentagree) in enumerate(current_percentagrees)
        averaged_percentagree_outputs[i] += c_percentagree / m_raters
    end
end


#THen all together, human and all GPT4 using m rater Fleiss Kappa
fleisskappa_outputs = []
for (c_var_type, col_human, col_GPT4_A, col_GPT4_B, col_GPT4_C, col_GPT4_D, col_GPT4_E, col_GPT4_F, col_GPT4_G, col_GPT4_H) in zip(var_types, Human_cols, values(GPT4_cols)...)
    c_var_data_df = df[:, [col_human, col_GPT4_A, col_GPT4_B, col_GPT4_C, col_GPT4_D, col_GPT4_E, col_GPT4_F, col_GPT4_G, col_GPT4_H]]

    c_var_data = Matrix(c_var_data_df)
    
    @rput c_var_data
    @rput c_var_type
    #dinfo1 = c_var_data
    R"c_var_data <- t(matrix(unlist(c_var_data), byrow=FALSE, ncol=9))"
    #R"dinfo2 <- c_var_data"
    #@rget dinfo2

    c_var_data = c_var_data_df[completecases(c_var_data_df), :] #Remove missin balues

    #push!(dinfo4, size(c_var_data))
    @rput c_var_data
    R"c_var_data <- t(matrix(unlist(c_var_data), byrow=FALSE, ncol=9))"
    weight_mode = "unweighted"
    @rput weight_mode

    R"fleisskappa_output <- kappam.fleiss(t(c_var_data), exact = FALSE, detail = FALSE)"
    @rget fleisskappa_output
    push!(fleisskappa_outputs, fleisskappa_output)
end

#Then average between all pairs of GPT4 annotators, without human
averaged_GPT4_cohenkappa_outputs, averaged_GPT4_percentagree_outputs = [0.0, 0.0],[0.0, 0.0]

for (GPT4_inst_label, GPT4_inst_cols) in GPT4_cols
    for (GPT4_inst_label2, GPT4_inst_cols2) in GPT4_cols
        if GPT4_inst_label != GPT4_inst_label2
            current_kappas, current_percentagrees = irr_2rater(GPT4_inst_cols, GPT4_inst_cols2)
            for (i, c_kappa) in enumerate(current_kappas)
                averaged_GPT4_cohenkappa_outputs[i] += c_kappa[:value] / (2*binomial(length(GPT4_cols), 2)) #the 2* because we are doing all pairs twice in the iteration
            end
            for (i, c_percentagree) in enumerate(current_percentagrees)
                averaged_GPT4_percentagree_outputs[i] += c_percentagree / (2*binomial(length(GPT4_cols), 2)) #the 2* because we are doing all pairs twice in the iteration
            end
        end
    end
end

In [40]:
@show cohenkappa_outputs
println("==========================")
@show percentagree_outputs
println("==========================")
@show averaged_cohenkappa_outputs
@show averaged_percentagree_outputs
println("==========================")
@show fleisskappa_outputs
println("==========================")
@show averaged_GPT4_cohenkappa_outputs
@show averaged_GPT4_percentagree_outputs

cohenkappa_outputs = Any[OrderedCollections.OrderedDict{Symbol, Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)", :subjects => 709, :raters => 2, :irr_name => "Kappa", :value => 0.8159276225176346, :stat_name => "z", :statistic => 21.74957827232842, :p_value => 0.0), OrderedCollections.OrderedDict{Symbol, Any}(:method => "Cohen's Kappa for 2 Raters (Weights: unweighted)", :subjects => 705, :raters => 2, :irr_name => "Kappa", :value => 0.8193198063454307, :stat_name => "z", :statistic => 28.935968674056202, :p_value => 0.0)]
percentagree_outputs = Any[91.67842031029619, 90.92198581560284]
averaged_cohenkappa_outputs = [0.7681912702297113, 0.7412164370045322]
averaged_percentagree_outputs = [89.41655359565807, 86.80461329715061]
fleisskappa_outputs = Any[OrderedCollections.OrderedDict{Symbol, Any}(:method => "Fleiss' Kappa for m Raters", :subjects => 737, :raters => 9, :irr_name => "Kappa", :value => 0.8505134465176729, :stat_name => "z", :statistic => 138.5371274845553

2-element Vector{Float64}:
 94.33998836983912
 91.76196937390964

#### ICR's full dataset (15%)

"Cohen's Kappa for 2 Raters Engagement (Human and GPT4 majority vote): <br />
:subjects => 709, <br />
:raters => 2, <br />
:irr_name => "Kappa", <br />
:value => 0.8159276225176346, <br />
:stat_name => "z", <br />
:statistic => 21.74957827232842, <br />
:p_value => 0.0 <br />

"Cohen's Kappa for 2 Raters Sentiment (Human and GPT4 majority vote): <br />
:subjects => 705,  <br />
:raters => 2,  <br />
:irr_name => "Kappa",  <br />
:value => 0.8193198063454307,  <br />
:stat_name => "z",  <br />
:statistic => 28.935968674056202,  <br />
:p_value => 0.0 <br />

percentagree_outputs: <br />
[Engagement = 91.67842031029619, Sentiment = 90.92198581560284] <br />


averaged_cohenkappa_outputs: <br />
[Engagement = 0.7681912702297113, Sentiment = 0.7412164370045322] <br />


"Fleiss' Kappa for m Raters Engagement": <br />
 :subjects => 737, <br />
 :raters => 9, <br />
 :irr_name => "Kappa", <br />
 :value => 0.8505134465176729, <br />
 :stat_name => "z",  <br />
 :statistic => 138.53712748455538,  <br />
 :p_value => 0.0 <br />
 
 "Fleiss' Kappa for m Raters Sentiment": <br />
 :subjects => 737,  <br />
 :raters => 9,  <br />
 :irr_name => "Kappa",  <br />
 :value => 0.8243488266877181, <br />
 :stat_name => "z",  <br />
 :statistic => 177.94930954638485,  <br />
 :p_value => 0.0 <br />


#### ICR's for the pilot:

"Cohen's Kappa for 2 Raters on engagement (Human and GPT4 majority vote), <br />
:subjects => 74, <br />
:raters => 2, <br />
:irr_name => "Kappa", <br />
:value => 0.6595092024539877, <br />
:p_value => 5.698373772844434e-9 <br />

"Cohen's Kappa for 2 Raters on sentiment (Human and GPT4 majority vote)" <br />
:subjects => 74 <br />
:raters => 2 <br />
:irr_name => "Kappa" <br />
:value => 0.6023150062009095 <br />
:statistic => 7.011686888701393 <br />
:p_value => 2.354560990625032e-12 <br />

percentagree_outputs: <br />
Engagement: 83.78378378378379, Sentiment: 82.43243243243244 <br />

averaged_cohenkappa_outputs: <br />
[Engagement: 0.6493745214862563, Sentiment: 0.5855066200229464] <br />


"Fleiss' Kappa for m Raters" <br />
:subjects => 75, <br />
:raters => 9, <br />
:irr_name => "Kappa", <br />
:value => 0.7858296636732497, <br />
:p_value => 0.0 <br />

"Fleiss' Kappa for m Raters" <br />
:subjects => 75, <br />
:raters => 9 <br />
:irr_name => "Kappa" <br />
:value => 0.8182602267126664, <br />
:stat_name => "z", <br />
:statistic => 56.55817001965667, <br />
:p_value => 0.0 <br />


In [30]:
#Compile outputs to simple dataframe:

code_numbers = replace.(String.(A_cols), "A." => "") 

irr_output_df = DataFrame(CodingPoint=code_numbers,
        Count = count_outputs,
        AgreementCount = agreementCount_outputs,
        PercentAgreeement=percentagree_outputs, 
        CohenKappa=[output[:value] for output in cohenkappa_outputs])
XLSX.writetable(dirpath*"output_table.xlsx", irr_output_df,  anchor_cell="B2")


UndefVarError: UndefVarError: A_cols not defined